### Selenium으로 50페이지

In [11]:
import os, warnings
import re
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import timeit

start = timeit.default_timer()
repeat = 50
### 물품 검색
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('./chromedriver_90')
url = 'https://www.amazon.com/'
driver.get(url)

sel_search = driver.find_element_by_xpath('//*[@id="twotabsearchtextbox"]')
sel_button = driver.find_element_by_xpath('//*[@id="nav-search-submit-button"]')

time.sleep(1)
sel_search.clear()
time.sleep(1)
sel_search.send_keys('earphone')
time.sleep(2)
sel_button.click()
time.sleep(3)

### 상품 목록 중 첫 번째 상품 선택
sel_a = driver.find_elements_by_xpath('//*[@id="search"]/div[1]/div/div[1]/div/span[3]/div[2]')

for i in sel_a:
    sel_b = i.find_elements_by_tag_name('h2')
    for j in sel_b:
        if j.text:
            try:
                print('제품명 : ', j.text)
                print('url : ', j.find_element_by_tag_name('a').get_attribute('href'))
                print()
                one_url = j.find_element_by_tag_name('a').get_attribute('href')
                break
            except:
                print('url : Fail')
                print()
                continue
    break
    
time.sleep(1)
driver.get(one_url)
time.sleep(3)

## 리뷰 창 띄우기
rating = driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]')
print(rating.text)
rating.click()
time.sleep(3)

## 모든 리뷰 보기 창
all_review = driver.find_element_by_xpath('//*[@id="reviews-medley-footer"]/div[2]/a')
all_review.click()
time.sleep(3)

reviews = []
names = []
stars = []
dates = []
countries = []
helpfuls = []

next_button = driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a')
time.sleep(1)
next_button.click()
time.sleep(2)
prev_button = driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[1]/a')
prev_button.click()
time.sleep(2)   # 22초

for i in range(repeat):    # 페이지 수 , 딜레이 시간(time.sleep) : 6 x 구동 횟수 + 22초
    next_button = driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a')
    time.sleep(2)
    
    print('=================')
    print('page', i+1, 'Crawling...')
    print('=================')
    idx1 = []
    idx2 = []
    idx3 = []
    
    for x_num in range(2, 12):
        xpath_1 = f'/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{x_num}]'
        review_list = driver.find_element_by_xpath(xpath_1)
        review_list = review_list.text.split('\n')
#         print(review_list)
        
        for idx, text in enumerate(review_list):
            if text[:6] == 'Color:' or text[:8] == 'Verified' or ('Reviewed' in text[:9]):
                idx1.append(idx+1)
            if 'people found' in text[:14] or 'One person f' in text[:14] or 'Helpful' in text[:14]:
                idx2.append(idx)
            if 'Reviewed' in text[:8]:
                idx3.append(idx)

        try:
            if idx1[x_num-1]:
                del idx1[x_num-2]
        except:
            pass
                
        try:
            if idx2[x_num-1]:
                del idx2[x_num-1]
        except:
            pass
        
        reviews.append('\n'.join(review_list[idx1[x_num-2]:idx2[x_num-2]]))
        
        if review_list[idx2[x_num-2]] == 'Helpful':
            helpfuls.append('No people found this helpful')
        else:
            helpfuls.append(review_list[idx2[x_num-2]])
        
        names.append(review_list[0])
        countries.append(review_list[idx3[x_num-2]].split(' on ')[0])
        dates.append(review_list[idx3[x_num-2]].split(' on ')[1])
        
    xpath_2 = '/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[5]/div[3]/div/div/div/div/div[2]/a[1]'
    score_list = driver.find_elements_by_xpath(xpath_2)
    
    for i in score_list : 
        stars.append(i.get_attribute('title')[:3])
        
    time.sleep(1)
    next_button.click()
    time.sleep(3)
    
print('================================')

print(len(names))
print(len(countries))
print(len(dates))
print(len(stars))
print(len(helpfuls))
print(len(reviews))

print('================================')

total_review = {'날짜':dates , '국가':countries , '이름':names , '평점':stars ,
                '리뷰':reviews , '도움':helpfuls}

total_review = pd.DataFrame(total_review)
stop = timeit.default_timer()

print('================================')
print('Time: ', stop - start - (6*repeat + 22), '(초)')
print('================================')

total_review

제품명 :  Amazon Basics In-Ear Wired Headphones Earbuds with Microphone, Red
url :  https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A049053719V83R4ZNWDQA&url=%2FAmazonBasics-17E13RD-Ear-Headphones-Mic%2Fdp%2FB07HH223KV%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dearphones%26qid%3D1624292070%26sr%3D8-1-spons%26psc%3D1&qualifier=1624292070&id=60513179276212&widgetName=sp_atf

6,565 ratings
page 1 Crawling...
page 2 Crawling...
page 3 Crawling...
page 4 Crawling...
page 5 Crawling...
page 6 Crawling...
page 7 Crawling...
page 8 Crawling...
page 9 Crawling...
page 10 Crawling...
page 11 Crawling...
page 12 Crawling...
page 13 Crawling...
page 14 Crawling...
page 15 Crawling...
page 16 Crawling...
page 17 Crawling...
page 18 Crawling...
page 19 Crawling...
page 20 Crawling...
page 21 Crawling...
page 22 Crawling...
page 23 Crawling...
page 24 Crawling...
page 25 Crawling...
page 26 Crawling...
page 27 Crawling...
page 28 Crawling...
page 29 C

,날짜,국가,이름,평점,리뷰,도움
0,"March 22, 2019",Reviewed in the United States,Rick Layton,2.0,I made a mistake. The original Earpods that ca...,No people found this helpful
1,"April 3, 2019",Reviewed in the United States,Emily,5.0,These are very inexpensive headphones but they...,No people found this helpful
2,"April 2, 2019",Reviewed in the United States,romeo,1.0,"The cables are very flimsy and thin, similar t...",No people found this helpful
3,"February 11, 2019",Reviewed in the United States,Ed W.,5.0,The Amazon basic ear headphones works in the i...,No people found this helpful
4,"February 28, 2019",Reviewed in the United States,Sunderrajan,4.0,I liked the color - my sons can't tell me that...,No people found this helpful
...,...,...,...,...,...,...
495,"January 28, 2019",Reviewed in the United States,ice grizzly,5.0,- No noise when rubbing against jacket/shirt\n...,3 people found this helpful
496,"March 20, 2019",Reviewed in the United States,Sophia11223,1.0,"At first they were great, then after like thre...",4 people found this helpful
497,"March 25, 2019",Reviewed in the United States,🌈 KT,3.0,"- Made in China\n- The cord is soft, flexible ...",No people found this helpful
498,"June 24, 2019",Reviewed in the United States,Sarah,5.0,I have been using these for several days and h...,One person found this helpful


In [13]:
total_review.to_csv("Amazon_Selenium_50.csv", index=False)

## 확인
print(os.getcwd())  # 현재 위치
print(os.listdir(os.getcwd()))

C:\Users\user\LikeLion_AI
['.ipynb_checkpoints', 'Amazon_beautifulSoup.ipynb', 'Amazon_BeautifulSoup_50.csv', 'Amazon_BeautifulSoup_L.ipynb', 'Amazon_review.csv', 'Amazon_review.ipynb', 'Amazon_selenium.csv', 'Amazon_selenium.ipynb', 'Amazon_Selenium_50.csv', 'Amazon_Selenium_L.ipynb', 'chromedriver_90.exe', 'data', 'dog_e.png', 'LLAI_06_09.ipynb', 'LLAI_06_14.ipynb', 'LLAI_06_16.ipynb', 'LLAI_06_17.ipynb', 'LLAI_06_18.ipynb', 'LLAI_06_21.ipynb', 'Untitled.ipynb', 'url.csv', 'Web_test.ipynb', '네이버 영화 정보.csv', '달라스바이어스클럽_리뷰_06_17.csv', '스파이더맨_리뷰_06_17.csv', '의료용어.csv', '인기종목 정보.csv', '회사명과 웹사이트.csv']


In [12]:
print(stop, start)

8125.6960762 7779.1408609
